In [1]:
from __future__ import division
import cPickle as pickle
import Queue
import time
import sys
sys.path.insert(0, './app/')

from HierarchicalPartitioningTree import PartitionNode, PartitionTree

from PartitionMethods import *

In [2]:
H = gt.load_graph('/media/parth/shared/workspace/Peel-Explorer/app/data/graphs/csx_06042017_peeled.gt')

In [3]:
# vp = H.new_vp('bool', vals=True)
# ep = H.new_ep('bool', vals=True)
# v_idx = np.where(vp.a == 1)[0]
# e_idx = np.where(ep.a == 1)[0]
# T = PartitionTree()
# T.root = PartitionNode(vertex_indices=v_idx,
#                        edge_indices=e_idx,
#                        partition_type='root',
#                        label='root',
#                        note='Overall graph, minus vertex peel 1')

In [4]:
# children = connected_components(H, vertex_indices=T.root.vertex_indices)
# T.root.children = children
# for child in T.root.children:
#     child.parent = T.root
#     # T.inverted_index[child.label] = child

In [5]:
# for idx, comp_child in enumerate(T.root.children):
#     v_idx = comp_child.vertex_indices
#     bicomp_children = biconnected_components(H, vertex_indices=v_idx)
#     comp_child.children = bicomp_children
#     for bicomp_child in comp_child.children:
#         bicomp_child.parent = comp_child
#         # T.inverted_index[bicomp_child.label] = bicom_child
#     if idx % 100 == 0 and idx > 0:
#         print idx

In [6]:
# for idx, bicomp_child in enumerate(T.root.children[0].children):
#     v_idx = bicomp_child.vertex_indices
#     if len(v_idx) < 100:
#         continue
#     epl_children = edge_peel(H, vertex_indices=v_idx)
#     bicomp_child.children = epl_children
#     for epl_child in bicomp_child.children:
#         epl_child.parent = bicomp_child
#     if idx % 100 == 0 and idx > 0:
#         print idx

# # for idx, bicomp_child in enumerate(T.root.children):
# #     v_idx = bicomp_child.vertex_indices
# #     epl_children = edge_peel(H, vertex_indices=v_idx)
# #     bicomp_child.children = epl_children
# #     for epl_child in bicomp_child.children:
# #         epl_child.parent = bicomp_child
# #         T.inverted_index[epl_child.label] = epl_child
# #     if idx % 100 == 0 and idx > 0:
# #         print idx

In [7]:
# for idx_c, comp_child in enumerate(T.root.children):
# #     comp_child.label += ('_' + str(idx_c))
#     print comp_child.label, comp_child.partition_type
#     if not comp_child.children:
#         continue
#     for idx_b, bicomp_child in enumerate(comp_child.children):
# #         bicomp_child.label = '|'.join([bicomp_child.parent.label, bicomp_child.label])
# #         bicomp_child.label = bicomp_child.label.split('|')[-1]
# #         bicomp_child.label += ('_' + str(idx_b))
#         bicomp_child.partition_type = 'edge'
#         print bicomp_child.label, bicomp_child.partition_type
#         if not bicomp_child.children:
#             continue
#         for idx_e, epl_child in enumerate(bicomp_child.children):
# #             epl_child.label = '|'.join([epl_child.parent.label, epl_child.label])
# #             epl_child.label = epl_child.label.upper()
# #             epl_child.label = epl_child.label.split('|')[-1]
# #             epl_child.label += ('_' + str(idx_e))
#             print epl_child.label, epl_child.partition_type

In [8]:
# with open('/home/parth/Desktop/tree_mod_min2.pkl', 'wb') as f:
#     pickle.dump(T, f)

In [9]:
# with open('/home/parth/Desktop/tree_mod_min.pkl', 'rb') as f:
#     T = pickle.load(f)

In [10]:
with open('app/data/trees/tree_mod_min3.pkl', 'rb') as f:
    T = pickle.load(f)

In [23]:
def collect_indices(root):
    Q = Queue.Queue()
    vlist = set()
    elist = set()
    Q.put(root)
    while Q.qsize() > 0:
        node = Q.get()
        if node.is_leaf():
            vlist.update(node.vertex_indices)
            elist.update(node.edge_indices)
        for child in node.children:
            Q.put(child)
    assert len(vlist) == root.num_vertices()
    assert len(elist) == root.num_edges()
    return list(vlist), list(elist)

In [12]:
def reached_stopping_criteria(node):
    if node.num_siblings() == 0:
        if node.parent is None:
            return False
        if node.parent.num_siblings() == 0:
            if node.parent.parent is None:
                return False
            return node.parent.parent.num_siblings() == 0
    return False

In [13]:
def determine_partition_method(node):
    short_label = node.label.split('|')[-1].upper()
    if short_label.startswith('CC'):
        operation = biconnected_components
    elif short_label.startswith('BCC'):
        operation = edge_peel
    elif short_label.startswith('EPL') or short_label == 'ROOT':
        operation = connected_components
    else:
        raise ValueError('Unknown Label')
    return operation

In [14]:
def explore_tree(G, root, threshold=256, check_partition=False):
    # TODO: Change this to be a stack. Not necessary, but probably better
    # Q = Queue.Queue()
    # Q.put(root)
    stack = []
    stack.append(root)
    # while Q.qsize() > 0:
    while len(stack) > 0:
        # node = Q.get()
        node = stack.pop()
        s = '{} --> |V|: {}, |E|: {}'.format(node.label,
                                             node.num_vertices(),
                                             node.num_edges())
        if not node.is_leaf():
            for child in node.children:
                # Q.put(child)
                stack.append(child)
            # print('Not a leaf')
            continue
        if node.num_vertices() < threshold:
            # print('V < threshold')
            continue
        if reached_stopping_criteria(node):
            print(s)
            print('Rock')
#             if node.num_vertices() < 1024:
#                 print('kcc')
#                 k_connected_components(G, node.vertex_indices, node.edge_indices)
            continue
        print(s)
        operation = determine_partition_method(node)
        vlist = node.vertex_indices
        elist = node.edge_indices
        children = operation(G, vertex_indices=vlist, edge_indices=elist)
        # this shouldn't happen
        if not children:
            # print('No children?')
            continue
        node.children = children
        v_part = set()
        e_part = set()
        for child in node.children:
            child.parent = node
            child.label = child.parent.label + '|' + child.label
            if check_partition:
                v_part.update(child.vertex_indices)
                e_part.update(child.edge_indices)
            # Q.put(child)
            stack.append(child)
        if check_partition:
            assert len(v_part) == node.num_vertices()
            assert len(e_part) == node.num_edges()
        node.vertex_indices = []
        node.edge_indices = []

In [15]:
# explore_tree(H, T.root, threshold=64, check_partition=False)

In [16]:
def traverse_bfs(root):
    Q = Queue.Queue()
    Q.put(root)
    v_hist = []
    e_hist = []
    while Q.qsize() > 0:
        node = Q.get()
        s = '{} --> |V|: {}, |E|: {}'.format(node.label,
                                             node.num_vertices(),
                                             node.num_edges())
        if node.is_leaf():
            v_hist.append(node.num_vertices())
            e_hist.append(node.num_edges())
            if node.num_vertices() > 512:
                print(s)
        for child in node.children:
            Q.put(child)
    return v_hist, e_hist

def traverse_dfs(root):
    stack = []
    stack.append(root)
    v_hist = []
    e_hist = []
    elist = []
    vlist = []
    idx = 0
    while len(stack) > 0:
        node = stack.pop()
        s = '{} --> |V|: {}, |E|: {}'.format(node.label,
                                             node.num_vertices(),
                                             node.num_edges())
        if node.is_leaf():
            v_hist.append(node.num_vertices())
            e_hist.append(node.num_edges())
            if node.label == 'root|CC_0_0|BCC_4325_4325|EPL_17_1|CC_1_1':
                print 'FOUND IT'
                vlist, elist = collect_indices(node)
                print len(vlist), len(elist)
            if node.num_vertices() > 128:
                idx += 1
                print(s)
#                 if node.num_vertices() < 512:
#                     print('kcc')
#                     k_connected_components(H, node.vertex_indices, node.edge_indices)
        for child in node.children[::-1]:
            stack.append(child)
    print idx
    return v_hist, e_hist, vlist, elist

In [17]:
v_hist, e_hist = traverse_dfs(T.root)
print len(v_hist), len(e_hist)
print np.sum(v_hist), np.sum(e_hist)
print T.root.num_vertices(), T.root.num_edges()

root|CC_0_0|BCC_4325_4325|EPL_18_0|CC_0_0|BCC_0_0|EPL_18_0|CC_0_0 --> |V|: 12983, |E|: 214907
FOUND IT
33 376
root|CC_0_0|BCC_4325_4325|EPL_15_3|CC_0_0|BCC_0_0|EPL_15_0 --> |V|: 5568, |E|: 72597
root|CC_0_0|BCC_4325_4325|EPL_14_4|CC_0_0|BCC_0_0|EPL_14_0|CC_0_0 --> |V|: 2073, |E|: 25708
root|CC_0_0|BCC_4325_4325|EPL_13_5|CC_0_0|BCC_0_0|EPL_13_0 --> |V|: 79379, |E|: 960404
root|CC_0_0|BCC_4325_4325|EPL_12_6|CC_0_0|BCC_2_2|EPL_12_0|CC_0_0|BCC_0_0|EPL_12_0 --> |V|: 581, |E|: 5921
root|CC_0_0|BCC_4325_4325|EPL_12_6|CC_0_0|BCC_3_3|EPL_12_0|CC_0_0|BCC_0_0 --> |V|: 1086, |E|: 10689
root|CC_0_0|BCC_4325_4325|EPL_10_8|CC_0_0|BCC_1_1|EPL_10_0|CC_0_0|BCC_0_0|EPL_10_0 --> |V|: 8537, |E|: 71255
root|CC_0_0|BCC_4325_4325|EPL_9_9|CC_0_0|BCC_11_11|EPL_9_0|CC_0_0|BCC_0_0|EPL_9_0 --> |V|: 69136, |E|: 537550
root|CC_0_0|BCC_4325_4325|EPL_8_10|CC_0_0|BCC_9_9|EPL_8_0|CC_0_0|BCC_0_0|EPL_8_0 --> |V|: 52029, |E|: 347536
root|CC_0_0|BCC_4325_4325|EPL_7_11|CC_0_0|BCC_16_16|EPL_7_0|CC_0_0|BCC_0_0|EPL_7_0 --> |V|:

ValueError: too many values to unpack

In [27]:
# Testing
vlist, elist = collect_indices(T.root.children[0].children[-1].children[1])
print len(vlist), len(elist)

101 1236


In [43]:
def create_adjacency_list(G, vlist, elist):
    vfilt = G.new_vp('bool', vals=False)
    vfilt.a[vlist] = True
    G.set_vertex_filter(vfilt)
    efilt = G.new_ep('bool', vals=False)
    efilt.a[elist] = True
    G.set_edge_filter(efilt)
    print G.num_vertices(), G.num_edges()
    for v in G.vertices():
        neighbors = [u for u in v.out_neighbours()]
        # First column for v; following columns are neighbors.
        # NOTE: Adjacency list is redundant for undirected graphs
        out_str = ('{} ' + 
                   ('{} ' * len(neighbors))[:-1] + 
                   '\n').format(v, *neighbors)
        print out_str

In [44]:
create_adjacency_list(H, vlist, elist)

101 1236
40727 2052027 288321 593178 2064585 1672068 1810813 569306 2052025 1400918 2052024 569308 1767861 587637 1923716 1437326 1697580 55526 154456 377173 796743 834558 876298 1329023 1491712 1819918 2244939

50161 1923716 593178 1275059 1697583 1697580 154456 569308 734919 796739 796742 796743 876298 906783 1163802 1458159 1552741 2052024 2052027 2176201 2212190 2244939

55526 796739 2238316 40727 2210575 1923716 587637 1552741 2052024 2100429 85810 2256722 1767861 2082369 1810813 2052027 2052026 377173 1163802

63722 414493 210148 308658 532360 577677 577678 758661 950040 1027464 1066613 1066614 1066617 1499152 1499153 1499154 1499156 1638447 1974930 1974932

71844 1923716 1275059 1957902 1810813 1400918 593178 1697580 154456 377173 796742 796743 834558 1163802 1329023 1458159 1491712 2052027 2062884 2176201 2210572 2212190

85810 2238316 587637 55526 154456 187642 529860 906783 1132904 1154613 1163802 1458159 1491712 1672068 1767861 1819918 2052027 2062884 2212190 2256722

154456

In [21]:
v_hist, e_hist, vlist, elist = traverse_dfs(T.root)
print len(v_hist), len(e_hist)

root|CC_0_0|BCC_4325_4325|EPL_18_0|CC_0_0|BCC_0_0|EPL_18_0|CC_0_0 --> |V|: 12983, |E|: 214907
FOUND IT
33 376
root|CC_0_0|BCC_4325_4325|EPL_15_3|CC_0_0|BCC_0_0|EPL_15_0 --> |V|: 5568, |E|: 72597
root|CC_0_0|BCC_4325_4325|EPL_14_4|CC_0_0|BCC_0_0|EPL_14_0|CC_0_0 --> |V|: 2073, |E|: 25708
root|CC_0_0|BCC_4325_4325|EPL_13_5|CC_0_0|BCC_0_0|EPL_13_0 --> |V|: 79379, |E|: 960404
root|CC_0_0|BCC_4325_4325|EPL_12_6|CC_0_0|BCC_2_2|EPL_12_0|CC_0_0|BCC_0_0|EPL_12_0 --> |V|: 581, |E|: 5921
root|CC_0_0|BCC_4325_4325|EPL_12_6|CC_0_0|BCC_3_3|EPL_12_0|CC_0_0|BCC_0_0 --> |V|: 1086, |E|: 10689
root|CC_0_0|BCC_4325_4325|EPL_10_8|CC_0_0|BCC_1_1|EPL_10_0|CC_0_0|BCC_0_0|EPL_10_0 --> |V|: 8537, |E|: 71255
root|CC_0_0|BCC_4325_4325|EPL_9_9|CC_0_0|BCC_11_11|EPL_9_0|CC_0_0|BCC_0_0|EPL_9_0 --> |V|: 69136, |E|: 537550
root|CC_0_0|BCC_4325_4325|EPL_8_10|CC_0_0|BCC_9_9|EPL_8_0|CC_0_0|BCC_0_0|EPL_8_0 --> |V|: 52029, |E|: 347536
root|CC_0_0|BCC_4325_4325|EPL_7_11|CC_0_0|BCC_16_16|EPL_7_0|CC_0_0|BCC_0_0|EPL_7_0 --> |V|:

In [22]:
efilt = H.new_ep('bool', vals=0)
efilt.a[list(elist)] = True
H.set_edge_filter(efilt)

In [23]:
print H.num_edges()

376
